# Library

In [ ]:
!pip install torchvision torchinfo

In [150]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import huggingface_hub
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModel,
    AutoModelForCausalLM,
    Trainer, TrainingArguments,
)
from torchvision.models import resnet18, ResNet18_Weights
from torchinfo import summary

# Transfer Learning

reference: [transfer learning](https://www.cse.ust.hk/~qyang/Docs/2009/tkde_transfer_learning.pdf)

<br>

<img src="https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-981-15-5971-6_83/MediaObjects/488258_1_En_83_Fig2_HTML.png" width="500">

<br>
<br>
<table border="1" cellpadding="5" cellspacing="0">
  <tr>
    <th colspan="3">Learning Settings</th>
    <th>Source and Target Domains</th>
    <th>Source and Target Tasks</th>
  </tr>
  <tr>
    <th colspan="3">Traditional Machine Learning</th>
    <td>the same</td>
    <td>the same</td>
  </tr>
  <tr>
    <td rowspan="3">Transfer Learning</td>
    <td colspan="2"><i>Inductive Transfer Learning \ Unsupervised Transfer Learning</i></td>
    <td>the same</td>
    <td>different but related</td>
  </tr>
  <tr>
    <td colspan="2"><i>Transfer Learning \ Unsupervised Transfer Learning</i></td>
    <td>different but related</td>
    <td>different but related</td>
  </tr>
  <tr>
    <td colspan="2"><i>Transductive Transfer Learning</i></td>
    <td>different but related</td>
    <td>the same</td>
  </tr>
</table>


<br>


| Transfer Learning Settings      | Related Areas                               | Source Domain Labels | Target Domain Labels | Tasks                       |
|---------------------------------|---------------------------------------------|----------------------|----------------------|-----------------------------|
| Inductive Transfer Learning     | Multi-task Learning                         | Available           | Available           | Regression, Classification  |
|             |  Self-taught Learning                                           | Unavailable         | Available           | Regression, Classification  |
| Transductive Transfer Learning  | Domain Adaptation, Sample Selection Bias, Co-variate Shift | Available           | Unavailable         | Regression, Classification  |
| Unsupervised Transfer Learning  |                                             | Unavailable         | Unavailable         | Clustering, Dimensionality Reduction |


# Fine Tuning

사전 훈련된 모델을 특정 작업이나 데이터셋에 맞게 조정하는 과정. <br>
대량의 데이터로 훈련되어 있는 pre-trained model을 특정 도메인이나 태스크에서 더 나은 성능을 내기 위해 모델의 weight를 조절하는 작업. <br>

In [2]:
# 사용할 모델 이름 설정
model_name = 'google-bert/bert-base-uncased'

# 지정한 모델 이름으로 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 지정한 모델 이름으로 시퀀스 분류 모델 초기화
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# IMDB 영화 리뷰 데이터셋을 로드
data = load_dataset('imdb')

# 학습 데이터셋에서 2000개의 샘플을 무작위로 섞어 선택
train = data['train'].shuffle(seed=0).select(range(2000))

# 테스트 데이터셋에서 1000개의 샘플을 무작위로 섞어 선택
test = data['test'].shuffle(seed=0).select(range(1000))

In [9]:
# 입력 예제를 토큰화하는 전처리 함수 정의
def preprocessing(example: str):
    return tokenizer(
        example['text'],          # 입력 텍스트를 가져옴
        padding='max_length',     # 최대 길이에 맞춰 패딩 추가
        max_length=256,           # 최대 길이를 256으로 설정
        truncation=True,          # 최대 길이를 초과할 경우 자르기
        return_tensors='pt',     # PyTorch 텐서 형태로 반환
    )

# 학습 데이터셋을 토큰화하여 새로운 데이터셋 객체 생성
tokenized_train_dataset = train.map(preprocessing, batched=True)

# 테스트 데이터셋을 토큰화하여 새로운 데이터셋 객체 생성
tokenized_test_dataset = test.map(preprocessing, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
# 레이블 비율 확인인
pd.Series(np.array([example.get('label') for example in tokenized_test_dataset])).value_counts()

0    507
1    493
Name: count, dtype: int64

In [16]:
class IMDBDataset(Dataset):
    def __init__(self, data):
        # 데이터셋 초기화: data는 토큰화된 데이터셋을 포함
        self.data = data
    
    def __len__(self):
        # 데이터셋의 크기(샘플 수)를 반환
        return len(self.data)
    
    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 데이터를 반환
        temp = self.data[idx]  # 인덱스에 해당하는 데이터 샘플을 가져옴

        return {
            'input_ids': temp.get('input_ids'),         # 입력 아이디
            'token_type_ids': temp.get('token_type_ids'), # 토큰 타입 아이디
            'attention_mask': temp.get('attention_mask'), # 어텐션 마스크
            'labels': temp.get('label')                    # 레이블 (감정 분류)
        }

# IMDBDataset 클래스의 인스턴스를 생성하여 학습 및 테스트 데이터셋을 만듭니다.
train_dataset = IMDBDataset(tokenized_train_dataset)  # 토큰화된 학습 데이터셋을 사용하여 train_dataset 생성
test_dataset = IMDBDataset(tokenized_test_dataset)    # 토큰화된 테스트 데이터셋을 사용하여 test_dataset 생성

In [25]:
# TrainingArguments 객체 생성: 모델 학습에 필요한 다양한 하이퍼파라미터 설정
training_args = TrainingArguments(
    output_dir='./results/bert',                 # 모델과 결과를 저장할 디렉토리 경로
    evaluation_strategy='epoch',                  # 검증 전략: 에포크마다 검증을 수행
    learning_rate=2e-5,                           # 학습률 설정
    per_device_train_batch_size=4,                # 학습 시 각 장치에서 사용할 배치 크기
    per_device_eval_batch_size=4,                 # 평가 시 각 장치에서 사용할 배치 크기
    num_train_epochs=1,                           # 학습할 에포크 수
    weight_decay=0.01,                            # 가중치 감쇠(정규화) 값
)

# Trainer 객체 생성: 모델 학습 및 평가를 위한 트레이너 인스턴스
trainer = Trainer(
    model=model,                                   # 학습할 모델
    args=training_args,                            # 설정된 학습 인자
    train_dataset=train_dataset,                   # 학습 데이터셋
    eval_dataset=test_dataset,                     # 평가 데이터셋
)

C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

## model

In [ ]:
parameters = list(model.parameters())

In [ ]:
## ff parameter
parameters[-8].shape

torch.Size([768, 3072])

In [ ]:
## ff parameter
parameters[-7].shape

torch.Size([768])

In [ ]:
# layernorm parameter
parameters[-6].shape

torch.Size([768])

In [ ]:
# layernorm parameter
parameters[-5].shape

torch.Size([768])

In [ ]:
# classifier weight
parameters[-4].shape

torch.Size([768, 768])

In [ ]:
# classifier bias
parameters[-3].shape

torch.Size([768])

In [ ]:
# classifier weight
parameters[-2]

Parameter containing:
tensor([[ 0.0248, -0.0028, -0.0064,  ...,  0.0203, -0.0279,  0.0060],
        [-0.0369, -0.0218, -0.0434,  ..., -0.0237,  0.0070, -0.0428]],
       requires_grad=True)

In [ ]:
# classifier bias
parameters[-1]

Parameter containing:
tensor([ 3.5986e-05, -3.5986e-05], requires_grad=True)

In [ ]:
# 모델의 모든 파라미터에 대해 학습을 하지 않도록 설정
for parameter in model.parameters():
    parameter.requires_grad = False  # 파라미터의 gradient 계산을 비활성화

# 모델의 모든 파라미터 중 마지막 파라미터를 리스트로 반환
list(model.parameters())[-1]  # gradient 없음

In [ ]:
# 사전 학습된 모델을 불러옵니다.
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 모델의 모든 파라미터에 대해 반복합니다.
for i, parameter in enumerate(list(model.parameters())):
    # 특정 인덱스(199, 200)의 파라미터만 학습 가능하도록 설정할 수 있는 주석 처리된 코드
    # if i in [199, 200]:
    #     parameter.requires_grad=True
    
    # 199 미만의 인덱스를 가진 파라미터의 기울기 계산을 비활성화
    if i < 199:
        parameter.requires_grad = False  # 해당 파라미터의 gradient 계산을 비활성화

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 모델 구조 확인
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# 사전 학습된 시퀀스 분류 모델을 불러옵니다.
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 모델의 모든 파라미터에 대해 이름과 함께 반복합니다.
for name, parameter in model.named_parameters():
    # 파라미터 이름에 'classifier'가 포함되어 있지 않은 경우
    if 'classifier' not in name:
        parameter.requires_grad = False  # 해당 파라미터의 gradient 계산을 비활성화

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 분류 레이어를 제외하고 모든 weight freeze
model = resnet18(pretrained=True)
model

In [ ]:
# 모델의 모든 파라미터에 대해 이름과 함께 반복합니다.
for name, parameter in model.named_parameters():
    # 파라미터 이름에 'fc'가 포함되어 있지 않은 경우
    if 'fc' not in name:
        parameter.requires_grad = False  # 해당 파라미터의 gradient 계산을 비활성화

## Discriminative Fine-tuning

paper: https://arxiv.org/pdf/1801.06146

모델의 서로다른 layer는 서로 다른 information을 학습할 것. <br>
-> layer마다 서로 다른 정도의 fine tuning이 필요. <br>

$ \eta_{L-1} = \frac{\eta_L}{2.6} $

<br> 

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> def configure_optimizers(self):
>     optimizer = optim.Adam(
>         {'params': self.model.classifier, 'lr': 0.001},
>         {'params': self.model.encoder.layer[-1], 'lr': 0.001/2.6},
>         {'params': self.model.encoder.layer[-2], 'lr': 0.001/2.6/2.6},
>     )
> ```

In [126]:
optim.Adam(model.parameters(), lr=2e-5)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 2e-05
    maximize: False
    weight_decay: 0
)

In [137]:
# Adam 옵티마이저를 설정하며, 각 레이어 및 파라미터에 대해 개별 학습률을 설정
optimizer = optim.Adam([
    # BERT의 임베딩 레이어 파라미터에 대한 학습률
    {'params': model.bert.embeddings.parameters(), 'lr': 2e-5/(2.6**14)},
    
    # BERT의 첫 번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[0].parameters(), 'lr': 2e-5/(2.6**13)},
    
    # BERT의 11번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-11].parameters(), 'lr': 2e-5/(2.6**12)},
    
    # BERT의 10번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-10].parameters(), 'lr': 2e-5/(2.6**11)},
    
    # BERT의 9번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-9].parameters(), 'lr': 2e-5/(2.6**10)},
    
    # BERT의 8번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-8].parameters(), 'lr': 2e-5/(2.6**9)},
    
    # BERT의 7번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-7].parameters(), 'lr': 2e-5/(2.6**8)},
    
    # BERT의 6번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-6].parameters(), 'lr': 2e-5/(2.6**7)},
    
    # BERT의 5번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-5].parameters(), 'lr': 2e-5/(2.6**6)},
    
    # BERT의 4번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-4].parameters(), 'lr': 2e-5/(2.6**5)},
    
    # BERT의 3번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-3].parameters(), 'lr': 2e-5/(2.6**4)},
    
    # BERT의 2번째 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-2].parameters(), 'lr': 2e-5/(2.6**3)},
    
    # BERT의 마지막 인코더 레이어 파라미터에 대한 학습률
    {'params': model.bert.encoder.layer[-1].parameters(), 'lr': 2e-5/(2.6**2)},
    
    # BERT의 풀러 레이어 파라미터에 대한 학습률
    {'params': model.bert.pooler.parameters(), 'lr': 2e-5/(2.6**1)},
    
    # 모델의 분류 레이어 파라미터에 대한 학습률
    {'params': model.classifier.parameters(), 'lr': 2e-5/(2.6**0)},
])

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 3.1002957437181844e-11
    maximize: False
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 8.060768933667281e-11
    maximize: False
    weight_decay: 0

Parameter Group 2
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 2.095799922753493e-10
    maximize: False
    weight_decay: 0

Parameter Group 3
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 5.449079799159082e-10
    maximize: False
    weight_decay: 0

Parameter Group 4
    amsgrad: False
    betas: (0.9, 0.999)
    capturable:

In [145]:
## bert layer의 learning rate를 수작업 없이 refactoring

# 파라미터 리스트를 초기화하고 BERT 임베딩 레이어의 파라미터를 추가
params = [{'params': model.bert.embeddings.parameters(), 'lr': 2e-5/(2.6**14)}]

# BERT 인코더 레이어의 파라미터를 추가 (12층부터 1층까지)
params.extend([
    {'params': model.bert.encoder.layer[-i].parameters(), 'lr': 2e-5/(2.6**(i+1))}
    for i in range(12, 0, -1)
])

# BERT 풀러 레이어와 분류 레이어의 파라미터를 추가
params.extend([
    {'params': model.bert.pooler.parameters(), 'lr': 2e-5/(2.6**1)},
    {'params': model.classifier.parameters(), 'lr': 2e-5/(2.6**0)},
])

## Slanted Triangular Learning Rates

<img src="https://production-media.paperswithcode.com/methods/new_lr_plot_tNtxBIM.jpg" width="400" height="300"/>

paper: https://arxiv.org/pdf/1801.06146

<br>

특정 task를 target으로 parameter를 조절하는 경우, 적합한 매개변수 영역으로 빠르게 수렴 후 업데이트하길 원함. <br>
이 경우 learning rate를 동일하게 사용하거나 점진적으로 감소하는 것이 바람직하지 않음. <br>
-> 학습률을 먼저 선형적으로 증가시킨 후, 선형적으로 감소시킴

$$

\text{cut} = \lfloor T \cdot \text{cut frac} \rfloor

\\

p =
\begin{cases} 
\frac{t}{\text{cut}}, & \text{if } t < \text{cut} \\
1 - \frac{t - \text{cut}}{\text{cut} \cdot (1/\text{cut frac} - 1)}, & \text{otherwise}
\end{cases}

\\ 

\eta_t = \eta_{\text{max}} \cdot \frac{1 + p \cdot (\text{ratio} - 1)}{\text{ratio}}

\\

\text{where T: num of iterations, cut frac: fraction of iterations}

$$

<br>

> ```python
> def configure_optimizers(self):
>     optimizer = optim.Adam(
>         self.model.parameters(),
>         lr=self.learning_rate,
>     )
> 
>     scheduler = {
>         'scheduler': optim.lr_scheduler.OneCycleLR(
>             optimizer,
>             max_lr=1e-2,
>             total_steps=1600,
>             pct_start=0.125,
>             anneal_strategy='linear'),
>         'interval': 'step',
>         'frequency': 1,
>     }
> 
>     return [optimizer], [scheduler]
> ```

In [124]:
optimizer = optim.Adam(model.parameters(), lr=0.1)

step = 300 # '<len(data)//batch>'
epoch = 2 # '<epoch>'

# OneCycleLR 스케줄러 설정
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=5e-4,               # 최대 학습률
    pct_start=0.15,            # 훈련의 처음 15% 동안 학습률을 증가
    total_steps=step * epoch    # 총 훈련 단계 수
)

## Gradual unfreezing

paper: https://arxiv.org/pdf/1801.06146

catastrophic forgetting 위험을 피하기 위해, 모델의 마지막 레이어부터 차례로 unfreeze하는 방법. <br>
마지막 layer는 가장 일반적인 지식을 포함하기에 마지막 레이어를 unfreeze 후 한 epoch 동안 fine-tuning. <br>
그 다음 layer를 unfreeeze 후 fine-tuning, 이 작업을 모든 layer가 unfreeze될 때까지 반복. <br>

<br>

<font style="font-size"> 사용 방법 </font>

<br>

> ```python
> def on_train_epoch_start(self):
>     self.n_unfreeze_layer += 1
>     for param in tuple(self.model.parameters())[-self.n_unfreeze_layer*2:]:
>         param.requires_grad = True
> ```

In [111]:
# 모델을 사전 훈련된 BERT 모델로 초기화
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 모든 파라미터의 gradient 업데이트를 비활성화
for name, parameter in model.named_parameters():
    parameter.requires_grad = False

# 에폭을 반복하며 특정 파라미터에 대해 gradient 업데이트를 활성화
for epoch in range(3):  # 3개의 에폭 동안 훈련
    for name, parameter in model.named_parameters():
        # 첫 번째 에폭 동안 classifier 계층의 파라미터 업데이트 활성화
        if epoch == 0 and 'classifier' in name:
            parameter.requires_grad = True
        
        # 두 번째 에폭 동안 pooler 계층의 파라미터 업데이트 활성화
        if epoch == 1 and 'pooler' in name:
            parameter.requires_grad = True
        
        # 2번째 에폭부터 14번째 에폭 전까지 특정 encoder layer의 파라미터 업데이트 활성화
        if 2 <= epoch < 14 and f'bert.encoder.layer.{13-epoch}' in name:
            parameter.requires_grad = True
        
        # 14번째 에폭 동안 embeddings 계층의 파라미터 업데이트 활성화
        if epoch == 14 and 'embedding' in name:
            parameter.requires_grad = True

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Practice

In [ ]:
huggingface-cli login

In [148]:
huggingface_hub.login()

In [151]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/llama-3.2-1B-Instruct')

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

c:\ProgramData\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
# llama LlamaDecoderLayer의 learning rate를 0.1부터해서 1/2씩 layer에 적용
# ex) LlamaDecoderLayer15: 0.1
#     LlamaDecoderLayer14: 0.1 * 1/2
#     ...
#     LlamaDecoderLayer0 : 0.1 * 1/2**x

optim.Adam(
    [{'params': model.model.layers[-i].parameters(), 'lr': 0.1 * (1/2**(i-1))} for i in range(1, 17)]
)